In [1]:
def load_sample_input():
    scores = [3, 7]
    elves = [0, 1]
    return scores, elves

In [2]:
scores, elves = load_sample_input()

In [3]:
def new_scores(scores, elves):
    digits = [scores[elve] for elve in elves]
    return list(int(item) for item in str(sum(digits)))

In [4]:
new_scores(scores, elves)

[1, 0]

In [5]:
scores.extend(new_scores(scores, elves))

In [6]:
scores

[3, 7, 1, 0]

In [7]:
def move_elves(scores, elves):
    moved = []
    for ind, elve in enumerate(elves):
        elve = (elve + scores[elve] + 1) % len(scores)
        moved.append(elve)
    return moved

In [8]:
move_elves(scores, elves)

[0, 1]

Let's run the simulation and print the output after each turn.

In [9]:
scores, elves = load_sample_input()
for _ in range(15):
    scores.extend(new_scores(scores, elves))
    elves = move_elves(scores, elves)
scores, elves

([3, 7, 1, 0, 1, 0, 1, 2, 4, 5, 1, 5, 8, 9, 1, 6, 7, 7, 9, 2], [8, 4])

In [10]:
len(scores)

20

Now all we need to do is extract the ten recipes.

In [11]:
def solve14(after):
    scores, elves = load_sample_input()
    for _ in range(after+10):
        scores.extend(new_scores(scores, elves))
        elves = move_elves(scores, elves)
    return "".join(map(str, scores[after:after+10]))

In [12]:
solve14(9)

'5158916779'

In [13]:
solve14(5)

'0124515891'

In [14]:
solve14(18)

'9251071085'

In [15]:
solve14(2018)

'5941429882'

In [16]:
solve14(598701)

'2776141917'

# Part 2 

Let's try the naive way.

In [17]:
from collections import deque

In [137]:
def solve14_part2(digits):
    scores, elves = load_sample_input()
    scores = list(scores)
    count = 0
    fullstr = "".join(map(str, scores[-10:]))
    while digits not in fullstr:
        new = new_scores(scores, elves)
        scores.extend(new)
        fullstr += "".join(map(str, new))
        elves = move_elves(scores, elves)
        count += 1
    return fullstr.index(digits)

In [138]:
solve14_part2('51589')

9

In [139]:
solve14_part2('01245')

5

In [140]:
solve14_part2('92510')

18

In [141]:
solve14_part2('59414')

2018

In [142]:
%timeit solve14_part2('59414')

8.4 ms ± 14.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [143]:
solve14_part2('598701')

KeyboardInterrupt: 

## a different way 

In [66]:
target = list(map(int, '598701'))
scores, elves = load_sample_input()
scores = list(scores)
count = 0
while True:
    new = new_scores(scores, elves)
    for recipe in new:
        scores.append(recipe)
    elves = move_elves(scores, elves)
    count += len(new)
    if count >= len(target):
        last = [scores[-1 - i] for i in range(len(target))]
        if ''.join(map(str, last[::-1])) == ''.join(map(str, target)):
            print('finished after: {}'.format(count - 3))
            break

KeyboardInterrupt: 

In [65]:
len(scores)

1530425

# Linked lists to the rescue 

I can only guess that the problem is due to the list that adds and adds up. Let's switch to linked lists, as in Problem 09 for example.

Elves now become nodes of the linked list of recipes.

In [69]:
import llist

In [133]:
def new_scores2(elves):
    digits = [elve.value for elve in elves]
    return list(int(item) for item in str(sum(digits)))

def move_elves2(scores, elves):
    moved = []
    for elve in elves:
        val = elve.value
        for _ in range(val + 1):
            elve = elve.next
            if elve is None:
                elve = scores.first
        moved.append(elve)
    return moved

target = list(map(int, '59414'))
scores, elves = load_sample_input()
scores = llist.dllist(scores)
elves = [scores.nodeat(0), scores.nodeat(1)]
count = 0
should_continue = True
while should_continue:
    new = new_scores2(elves)
    for recipe in new:
        scores.append(recipe)
        count += 1
        if count >= len(target):
            last_five = [scores.last]
            for _ in range(4):
                last_five.append(last_five[-1].prev)
            last = [node.value for node in last_five]
            if last[::-1] == target:
                print('finished after: {}'.format(count - 3), elves)
                should_continue = False
                break
    elves = move_elves2(scores, elves)

finished after: 2018 [<dllistnode(8)>, <dllistnode(6)>]
